In [1]:
# Data Science
import pandas as pd
from useful import *

# Others
from tqdm.notebook import tqdm
tqdm.pandas()
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data(longitude, latitude, season, ha, settings):
    data_dic = {}
    for sat in settings["assests"]:
        for band in settings["assests"][sat]:
            data_dic[sat + " " + band] = []
    data_dic["sentinel 1 vv/vh"] = []
    data_dic["sentinel 1 rvi"] = []
    
    data_dic["sentinel 2 ndvi"] = []
    data_dic["sentinel 2 rdvi"] = []
    data_dic["sentinel 2 gndvi"] = []
    data_dic["sentinel 2 savi"] = []
    data_dic["sentinel 2 evi"] = []
    data_dic["sentinel 2 tvi"] = []
    data_dic["sentinel 2 ari1"] = []
    data_dic["sentinel 2 ari2"] = []
    data_dic["sentinel 2 cri1"] = []
    data_dic["sentinel 2 cri2"] = []
    
    data_dic["landsat ndvi"] = []
    data_dic["landsat temperature"] = []
    
    data_dic["sentinel 1 date"] = []
    data_dic["sentinel 2 date"] = []
    data_dic["landsat date"] = []
    
    
    if season == 'SA':
        time_slice = "2022-05-01/2022-08-31"
    if season == 'WS':
        time_slice = "2022-01-01/2022-04-30"
    detection_time_slice = "2022-01-01/2022-12-31"
    
    bbox_size = settings["bbox_size"] * ha
    bbox_of_interest = [longitude-bbox_size, latitude-bbox_size, longitude+bbox_size, latitude+bbox_size]
    
    items_sentinel_1 = list(settings["catalog"].search(collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_slice).item_collection())
    items_sentinel_2 = list(settings["catalog"].search(collections=["sentinel-2-l2a"], bbox=bbox_of_interest, datetime=time_slice, query={"eo:cloud_cover": {"lt": 10}}).item_collection())
    items_landsat = list(settings["catalog"].search(collections=["landsat-c2-l2"], bbox=bbox_of_interest, datetime=time_slice, query={
        "eo:cloud_cover": {"lt": 10},
        "platform": {"in": ["landsat-8", "landsat-9"]},
    }).item_collection())
    #detection_matrix = get_detection_matrix(bbox_of_interest, settings["catalog"], detection_time_slice, settings)

    for item in items_sentinel_1:
        data = stac_load([item], bands=settings["assests"]["sentinel 1"], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
        if(data['vh'].values[0][0]!=-32768.0 and data['vv'].values[0][0]!=-32768.0):
            data = data.where(~data.isnull(), 0)
            
            vh = data["vh"].astype("float64")# * detection_matrix
            vv = data["vv"].astype("float64")# * detection_matrix
            
            vv_med = np.nanmedian(vv)
            vh_med = np.nanmedian(vh)
            
            data_dic["sentinel 1 vv"].append(vv_med)
            data_dic["sentinel 1 vh"].append(vh_med)
            data_dic["sentinel 1 vv/vh"].append(vv_med/vh_med)
            data_dic["sentinel 1 rvi"].append((np.sqrt((vv_med / (vv_med + vh_med))))*((4*vh_med)/(vv_med + vh_med)))
            
            data_dic["sentinel 1 date"].append(data.time.to_numpy())
            
            
    for item in items_sentinel_2:
        data = stac_load([item], bands=settings["assests"]["sentinel 2"], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
        cloud_mask = \
            (data.SCL != 0) & \
            (data.SCL != 1) & \
            (data.SCL != 3) & \
            (data.SCL != 6) & \
            (data.SCL != 8) & \
            (data.SCL != 9) & \
            (data.SCL != 10) 
        data = data.where(cloud_mask)
        for band in settings["assests"]["sentinel 2"]:
            data_dic["sentinel 2 " + band].append(np.nanmedian(data[band].astype("float64")))# * detection_matrix))
            
        nir = np.nanmedian(data["nir"].astype("float64"))# * detection_matrix)
        red = np.nanmedian(data["red"].astype("float64"))# * detection_matrix)
        green = np.nanmedian(data["green"].astype("float64"))# * detection_matrix)
        blue = np.nanmedian(data["blue"].astype("float64"))# * detection_matrix)

        data_dic["sentinel 2 ndvi"].append((nir - red) / (nir + red))
        data_dic["sentinel 2 rdvi"].append((nir - red) / np.sqrt(nir + red))
        data_dic["sentinel 2 gndvi"].append((nir - green) / (nir + green))
        data_dic["sentinel 2 savi"].append((nir - red) / (nir + red + 0.5) * (1.0 + 0.5))
        data_dic["sentinel 2 evi"].append(2.5 * ((nir - red) / (nir + (6 * red) - (7.5 * blue) + 1)))
        data_dic["sentinel 2 tvi"].append(np.sqrt(((nir - red) / (nir + red)) + 0.5))
        data_dic["sentinel 2 ari1"].append((1/green) - (1/nir))
        data_dic["sentinel 2 ari2"].append(nir * ((1/green) - (1/nir)))
        data_dic["sentinel 2 cri1"].append((1/blue) - (1/green))
        data_dic["sentinel 2 cri2"].append((1/blue) - (1/nir))
            
        data_dic["sentinel 2 date"].append(data.time.to_numpy())
            
    for item in items_landsat:
        try:
            data = stac_load([item], bands=settings["assests"]["landsat"], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
            my_mask = get_mask_landsat(data['qa_pixel'],
                       ['fill', 'dilated_cloud', 'cirrus', 
                        'cloud', 'shadow', 'water'],
                        settings["bit_flags"])
            my_mask = my_mask.to_numpy()
            my_mask[~my_mask] = np.nan
            for band in settings["assests"]["landsat"]:            
                data_dic["landsat " + band].append(np.nanmedian(data[band].astype("float64") * my_mask))

            nir = np.nanmedian(data["nir08"].astype("float64").to_numpy() * my_mask)
            red = np.nanmedian(data["red"].astype("float64").to_numpy() * my_mask)
            data_dic["landsat ndvi"].append((nir - red) / (nir + red))

            band_info = item.assets["lwir11"].extra_fields["raster:bands"][0]
            temperature = data["lwir11"].astype("float64")  * my_mask
            temperature *= band_info["scale"]
            temperature += band_info["offset"]
            data_dic["landsat temperature"].append(np.nanmedian(temperature.to_numpy()))

            data_dic["landsat date"].append(data.time.to_numpy())
        except:
            for band in settings["assests"]["landsat"]:            
                data_dic["landsat " + band].append(np.nan)
                data_dic["landsat ndvi"].append(np.nan)
                data_dic["landsat temperature"].append(np.nan)
            data_dic["landsat date"].append(np.nan)
                    
    return data_dic

def dic_to_df(dic):
    rows = []
    for d in dic:
        rows.append(pd.Series(d))
    return pd.concat(rows, axis=1).T

def get_mask_landsat(mask, flags_list, bit_flags):
    
    # Create the result mask filled with zeros and the same shape as the mask
    final_mask = np.zeros_like(mask)
    
    # Loop through the flags  
    for flag in flags_list:
        
        # get the mask for each flag
        flag_mask = np.bitwise_and(mask, bit_flags[flag])
        
        # add it to the final flag
        final_mask = final_mask | flag_mask
    
    return final_mask > 0

In [3]:
settings = {
    "bbox_size" : 0.0005,
    "data_name" : "8",
    "assests" : {"sentinel 1" : ['vh','vv'],
                 "sentinel 2" : ["red", "green", "blue", "nir", "SCL", "B05", "B06", "B07", "B8A"],
                 "landsat" : ["nir08", "red", "green", "blue", "qa_pixel", "lwir11"]
                },
    "bit_flags" : {
            'fill': 1<<0,
            'dilated_cloud': 1<<1,
            'cirrus': 1<<2, 
            'cloud': 1<<3,
            'shadow': 1<<4, 
            'snow': 1<<5, 
            'clear': 1<<6,
            'water': 1<<7
            },
    "catalog": pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
}

In [4]:
crop_yield_data = pd.read_csv("Crop_Yield_Data_challenge_2.csv")
test_file = pd.read_csv('Challenge_2_submission_template.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Challenge_2_submission_template.csv'

In [6]:
## Get Sentinel-1-RTC Data
data = crop_yield_data.progress_apply(lambda x: get_data(x['Longitude'], x['Latitude'],x['Season(SA = Summer Autumn, WS = Winter Spring)'], x["Field size (ha)"], settings), axis=1)
dic_to_df(data).to_csv("data_" + settings["data_name"] + ".csv", index=False)

  0%|          | 0/557 [00:00<?, ?it/s]

In [7]:
## Get Sentinel-2 Data
data_sub=test_file.progress_apply(lambda x: get_data(x['Longitude'], x['Latitude'],x['Season(SA = Summer Autumn, WS = Winter Spring)'], x["Field size (ha)"], settings), axis=1)
dic_to_df(data_sub).to_csv("submission_data_" + settings["data_name"] + ".csv", index=False)

  0%|          | 0/100 [00:00<?, ?it/s]